# 03 - Feature Engineering

Create and encode features for modeling.

**Steps**:
1. Create financial ratios
2. One-Hot encode categorical variables
3. Label encode high-cardinality categoricals
4. Prepare final feature set

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Paths
INTERIM_DATA = Path("../data/interim/lending_club_cleaned.parquet")
PROCESSED_DATA = Path("../data/processed")
PROCESSED_DATA.mkdir(parents=True, exist_ok=True)

In [ ]:
# Load cleaned data
df = pd.read_parquet(INTERIM_DATA)
print(f"Loaded data shape: {df.shape}")

## 1. Create Financial Ratios

Domain-specific features that are meaningful for credit risk.

In [ ]:
# Loan-to-Income Ratio
# How much of annual income is the loan amount?
if 'loan_amnt' in df.columns and 'annual_inc' in df.columns:
    df['loan_to_income'] = df['loan_amnt'] / df['annual_inc']
    # Cap extreme values
    df['loan_to_income'] = df['loan_to_income'].clip(upper=df['loan_to_income'].quantile(0.99))
    print(f"loan_to_income: mean={df['loan_to_income'].mean():.3f}, max={df['loan_to_income'].max():.3f}")

In [ ]:
# Payment Burden (Annual Payment / Annual Income)
# What percentage of income goes to loan payments?
if 'installment' in df.columns and 'annual_inc' in df.columns:
    df['payment_to_income'] = (df['installment'] * 12) / df['annual_inc']
    df['payment_to_income'] = df['payment_to_income'].clip(upper=df['payment_to_income'].quantile(0.99))
    print(f"payment_to_income: mean={df['payment_to_income'].mean():.3f}")

In [ ]:
# High Utilization Flag
# Credit utilization > 80% is a risk factor
if 'revol_util' in df.columns:
    df['high_utilization'] = (df['revol_util'] > 80).astype(int)
    print(f"high_utilization: {df['high_utilization'].mean():.1%} of borrowers")

In [ ]:
# DTI Risk Category
if 'dti' in df.columns:
    df['dti_risk'] = pd.cut(
        df['dti'], 
        bins=[-np.inf, 10, 20, 35, np.inf],
        labels=['low', 'moderate', 'high', 'very_high']
    )
    print("DTI Risk distribution:")
    print(df['dti_risk'].value_counts())

In [ ]:
# Income Category
if 'annual_inc' in df.columns:
    df['income_category'] = pd.cut(
        df['annual_inc'],
        bins=[0, 30000, 60000, 100000, 200000, np.inf],
        labels=['low', 'lower_middle', 'middle', 'upper_middle', 'high']
    )
    print("Income category distribution:")
    print(df['income_category'].value_counts())

In [ ]:
# Log transformations for skewed distributions
if 'annual_inc' in df.columns:
    df['log_annual_inc'] = np.log1p(df['annual_inc'])
    
if 'revol_bal' in df.columns:
    df['log_revol_bal'] = np.log1p(df['revol_bal'])

print("Added log transformations for skewed features")

## 2. Encode Categorical Variables

In [ ]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"Categorical columns ({len(categorical_cols)}):")
for col in categorical_cols:
    n_unique = df[col].nunique()
    print(f"  {col}: {n_unique} unique values")

In [ ]:
# One-Hot Encoding for low-cardinality categoricals
low_card_cols = ['term', 'grade', 'home_ownership', 'verification_status', 
                 'purpose', 'application_type', 'initial_list_status',
                 'dti_risk', 'income_category']

# Filter to existing columns
low_card_cols = [c for c in low_card_cols if c in df.columns]
print(f"One-Hot encoding columns: {low_card_cols}")

df_encoded = pd.get_dummies(df, columns=low_card_cols, drop_first=True)
print(f"Shape after One-Hot encoding: {df_encoded.shape}")

In [ ]:
# Label Encoding for high-cardinality categoricals (sub_grade)
if 'sub_grade' in df_encoded.columns:
    le = LabelEncoder()
    df_encoded['sub_grade_encoded'] = le.fit_transform(df_encoded['sub_grade'].astype(str))
    df_encoded = df_encoded.drop(columns=['sub_grade'])
    print(f"Label encoded sub_grade: {df_encoded['sub_grade_encoded'].nunique()} values")

In [ ]:
# Drop remaining object columns (text that can't be encoded meaningfully)
remaining_object_cols = df_encoded.select_dtypes(include=['object']).columns.tolist()
if remaining_object_cols:
    print(f"Dropping remaining object columns: {remaining_object_cols}")
    df_encoded = df_encoded.drop(columns=remaining_object_cols, errors='ignore')

In [ ]:
# Drop original emp_length (we have emp_length_numeric)
if 'emp_length' in df_encoded.columns:
    df_encoded = df_encoded.drop(columns=['emp_length'])

## 3. Final Cleanup

In [ ]:
# Handle any remaining missing values
missing_count = df_encoded.isnull().sum().sum()
if missing_count > 0:
    print(f"Filling {missing_count} remaining missing values with median")
    df_encoded = df_encoded.fillna(df_encoded.median())
else:
    print("No missing values")

In [ ]:
# Handle infinite values
df_encoded = df_encoded.replace([np.inf, -np.inf], np.nan)
df_encoded = df_encoded.fillna(df_encoded.median())

In [ ]:
# Final data types check
print("\nData types:")
print(df_encoded.dtypes.value_counts())

In [ ]:
# Final summary
print("\n" + "="*50)
print("FEATURE ENGINEERING SUMMARY")
print("="*50)
print(f"Final shape: {df_encoded.shape}")
print(f"Number of features: {df_encoded.shape[1] - 1}")  # Exclude target
print(f"Target: 'default'")
print(f"Default rate: {df_encoded['default'].mean():.2%}")

In [ ]:
# List all features
features = [c for c in df_encoded.columns if c != 'default']
print(f"\nFeatures ({len(features)}):")
for i, feat in enumerate(features, 1):
    print(f"{i:3}. {feat}")

## 4. Save Processed Data

In [ ]:
# Save to processed folder
output_path = PROCESSED_DATA / "lending_club_processed.parquet"
df_encoded.to_parquet(output_path, index=False)

import os
file_size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f"Saved to: {output_path}")
print(f"File size: {file_size_mb:.1f} MB")

## Next Steps

Proceed to `04_eda.ipynb` for:
- Exploratory Data Analysis
- Visualizations
- Correlation analysis